In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from skimage.io import imread
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from skimage.morphology import label

import gc
gc.enable()

In [ ]:
# Train and test directories
train_image_dir = "E:/train_v2"
test_image_dir = "E:/test_v2"

In [ ]:
# Getting into train directory
train_images = os.listdir(train_image_dir)
train_images.sort()
print(f"Total of {len(train_images)} images in train directory.\nHere is how first five train_images looks like:- {train_images[:5]}")

In [ ]:
# Train ships segmented masks
masks = pd.read_csv('C:/Users/krish/Downloads/train_ship_segmentations_v2.csv')


In [ ]:
# Define functions to do these tasks for all the training images
def rle_decode(mask_rle, shape=(768,768)):
    '''
    Input arguments -
    mask_rle: Mask of one ship in the train image
    shape: Output shape of the image array
    '''
    s = mask_rle.split()                                                               # Split the mask of each ship that is in RLE format
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]     # Get the start pixels and lengths for which image has ship
    ends = starts + lengths - 1                                                        # Get the end pixels where we need to stop
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)                                  # A 1D vec full of zeros of size = 768*768
    for lo, hi in zip(starts, ends):                                                   # For each start to end pixels where ship exists
        img[lo:hi+1] = 1                                                               # Fill those values with 1 in the main 1D vector
    '''
    Returns -
    Transposed array of the mask: Contains 1s and 0s. 1 for ship and 0 for background
    '''
    return img.reshape(shape).T                                                       

def masks_as_image(in_mask_list):
    '''
    Input - 
    in_mask_list: List of the masks of each ship in one whole training image
    '''
    all_masks = np.zeros((768, 768), dtype = np.int16)                                 # Creating 0s for the background
    for mask in in_mask_list:                                                          # For each ship rle data in the list of mask rle 
        if isinstance(mask, str):                                                      # If the datatype is string
            all_masks += rle_decode(mask)                                              # Use rle_decode to create one mask for whole image
    '''
    Returns - 
    Full mask of the training image whose RLE data has been passed as an input
    '''
    return np.expand_dims(all_masks, -1)

In [ ]:
'''Note that NaN values in the EncodedPixels are of float type and everything else is a string type'''   

# Add a new feature to the masks data frame named as ship. If Encoded pixel in any row is a string, there is a ship else there isn't. 
masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
masks.head(9)

In [ ]:
# Making a new data frame with unique image ids where we are summing up the ship counts
unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index() 
unique_img_ids.index+=1 # Incrimenting all the index by 1
unique_img_ids.head()

In [ ]:
# Adding two new features to unique_img_ids data frame. If ship exists in image, val is 1 else 0. And it's vec form
unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)
unique_img_ids.head()

In [ ]:
# Check the size of the files. Will take some time to run as there are loads of files!!!
unique_img_ids['file_size_kb'] = unique_img_ids['ImageId'].map(lambda c_img_id: os.stat(os.path.join(train_image_dir, c_img_id)).st_size/1024)
'''os.stat is used to get status of the specified path. Here, st_size represents size of the file in bytes. Converting it into kB!'''

In [ ]:
# We can get rid of any images whose size is less than 35 Kb. As some of the files are corrupted! 
unique_img_ids[unique_img_ids.file_size_kb<35].head()

In [ ]:
# Keep the files whose size > 35 kB
unique_img_ids = unique_img_ids[unique_img_ids.file_size_kb > 35]
unique_img_ids.head()

In [ ]:
# Also, retrive the old masks data frame
masks.drop(['ships'], axis=1, inplace=True)
masks.index+=1 
masks.head()

In [ ]:
# Train - Test split
from sklearn.model_selection import train_test_split                   
train_ids, valid_ids = train_test_split(unique_img_ids, test_size = 0.3, stratify = unique_img_ids['ships'])

In [ ]:
# Create train data frame
train_df = pd.merge(masks, train_ids)

# Create test data frame
valid_df = pd.merge(masks, valid_ids)

In [ ]:
print("There are ~")
print(train_df.shape[0], 'training masks,')
print(valid_df.shape[0], 'validation masks.')

In [ ]:
# Clipping the max value of grouped_ship_count to be 7, minimum to be 0
train_df['grouped_ship_count'] = train_df.ships.map(lambda x: (x+1)//2).clip(0,7)

In [ ]:
# Check
train_df.grouped_ship_count.value_counts()

In [ ]:
# Random Under-Sampling ships
def sample_ships(in_df, base_rep_val=1500):
    '''
    Input Args:
    in_df - dataframe we want to apply this function
    base_val - random sample of this value to be taken from the data frame
    '''
    if in_df['ships'].values[0]==0:                                                 
        return in_df.sample(base_rep_val//3)  # Random 1500//3 = 500 samples taken whose ship count is 0 in an image 
    else:                                 
        return in_df.sample(base_rep_val)    # Random 1500 samples taken whose ship count is not 0 in an image

In [ ]:
# Creating groups of ship counts and applying the sample_ships functions to randomly undersample the ships
balanced_train_df = train_df.groupby('grouped_ship_count').apply(sample_ships)
balanced_train_df.grouped_ship_count.value_counts() # In each group we have total of 1500 ships except 0 as we have decreased it even more to 500

In [ ]:
# Explaining what we just did if still not clear
for i in range(8):
    df_val_counts = balanced_train_df[balanced_train_df.grouped_ship_count==i].ships.value_counts()
    print(f"Data frame for grouped ship count = {i}:-\n{df_val_counts}\nSum of Values:- {df_val_counts.values.sum()}\n\n")


In [ ]:
!pip install -U -q segmentation-models
!pip install -q tensorflow==2.2.1
!pip install -q keras==2.5
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.util import montage
from skimage.segmentation import mark_boundaries

from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
import keras.backend as K

import segmentation_models as sm

# Parameters
BATCH_SIZE = 4
EDGE_CROP = 16
NB_EPOCHS = 100
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
NET_SCALING = None
IMG_SCALING = (1, 1)
VALID_IMG_COUNT = 400
MAX_TRAIN_STEPS = 20
BACKBONE = 'resnet50'

In [ ]:
# Image and Mask Generator
def make_image_gen(in_df, batch_size = BATCH_SIZE):
    '''
    Inputs -
    in_df - data frame on which the function will be applied
    batch_size - number of training examples in one iteration
    '''
    all_batches = list(in_df.groupby('ImageId'))                             # Group ImageIds and create list of that dataframe
    out_rgb = []                                                             # Image list
    out_mask = []                                                            # Mask list
    while True:                                                              # Loop for every data
        np.random.shuffle(all_batches)                                       # Shuffling the data
        for c_img_id, c_masks in all_batches:                                # For img_id and msk_rle in all_batches
            rgb_path = os.path.join(train_image_dir, c_img_id)               # Get the img path
            c_img = imread(rgb_path)                                         # img array
            c_mask = masks_as_image(c_masks['EncodedPixels'].values)         # Create mask of rle data for each ship in an img
            out_rgb += [c_img]                                               # Append the current img in the out_rgb / img list
            out_mask += [c_mask]                                             # Append the current mask in the out_mask / mask list
            if len(out_rgb)>=batch_size:                                     # If length of list is more or equal to batch size then
                yield np.stack(out_rgb)/255.0, np.stack(out_mask)            # Yeild the scaled img array (b/w 0 and 1) and mask array (0 for bg and 1 for ship)
                out_rgb, out_mask=[], []                                     # Empty the lists to create another batch

In [ ]:
# Generate train data 
train_gen = make_image_gen(balanced_train_df)

# Image and Mask
train_x, train_y = next(train_gen)

# Print the summary
print(f"train_x ~\nShape: {train_x.shape}\nMin value: {train_x.min()}\nMax value: {train_x.max()}")
print(f"\ntrain_y ~\nShape: {train_y.shape}\nMin value: {train_y.min()}\nMax value: {train_y.max()}")

In [ ]:
# Visulaising train batch
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
batch_rgb = montage_rgb(train_x)                                                   # Create montage of img
batch_seg = montage(train_y[:, :, :, 0])                                           # Create montafe of msk
batch_overlap = mark_boundaries(batch_rgb, batch_seg.astype(int))                  # Create bounding box around ships in img
titles = ["Images", "Segmentations", "Bounding Boxes on ships in Images"]          # Titles for subplot
colors = ['g', 'm', 'b']                                                           # Colors to be used for title
display = [batch_rgb, batch_seg, batch_overlap]                                    # What to display in subplot
plt.figure(figsize=(25,10))                                                        # Generate figure 
for i in range(3):                                                                 # For i = 0, 1, 2, 3                           
    plt.subplot(1, 3, i+1)                                                         # Create subplot
    plt.imshow(display[i])                                                         # Display 
    plt.title(titles[i], fontsize = 18, color = colors[i])                         # Title 
    plt.axis('off')                                                                # Turn off the axis
plt.suptitle("Batch Visualizations", fontsize = 20, color = 'r', weight = 'bold')  # Add suptitle
plt.tight_layout()                                                                 # Layout for subplot

In [ ]:
# Prepare validation data
valid_x, valid_y = next(make_image_gen(valid_df, VALID_IMG_COUNT))
print(f"valid_x ~\nShape: {valid_x.shape}\nMin value: {valid_x.min()}\nMax value: {valid_x.max()}")
print(f"\nvalid_y ~\nShape: {valid_y.shape}\nMin value: {valid_y.min()}\nMax value: {valid_y.max()}")

In [ ]:
# Augmenting Data using ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preparing image data generator arguments
dg_args = dict(rotation_range = 15,            # Degree range for random rotations
               horizontal_flip = True,         # Randomly flips the inputs horizontally
               vertical_flip = True,           # Randomly flips the inputs vertically
               data_format = 'channels_last')  # channels_last refer to (batch, height, width, channels)

In [ ]:
image_gen = ImageDataGenerator(**dg_args)
label_gen = ImageDataGenerator(**dg_args)

def create_aug_gen(in_gen, seed = None):
    '''
    Takes in -
    in_gen - train data generator, seed value
    '''
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))  # Randomly assign seed value if not provided
    for in_x, in_y in in_gen:                                                    # For imgs and msks in train data generator
        seed = 12                                                                # Seed value for imgs and msks must be same else augmentation won't be same
        
        # Create augmented imgs
        g_x = image_gen.flow(255*in_x,                                           # Inverse scaling on imgs for augmentation                                       
                             batch_size = in_x.shape[0],                         # batch_size = 3
                             seed = seed,                                        # Seed
                             shuffle=True)                                       # Shuffle the data
        
        # Create augmented masks
        g_y = label_gen.flow(in_y,
                             batch_size = in_x.shape[0],                       
                             seed = seed,                                         
                             shuffle=True)                                       
        
        '''Yeilds - augmented scaled imgs and msks array'''
        yield next(g_x)/255.0, next(g_y)

In [ ]:
# Augment the train data
cur_gen = create_aug_gen(train_gen, seed = 42)
t_x, t_y = next(cur_gen)
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())

In [ ]:
# Preprocess input for ResNet50
preprocess_input = sm.get_preprocessing(BACKBONE)
train_x = preprocess_input(train_x)
valid_x = preprocess_input(valid_x)

In [ ]:
# BACKBONE = 'resnet50'
# resnet50_unet = sm.Unet(BACKBONE, input_shape=(None, None, 3), encoder_weights='imagenet', classes=1, activation='sigmoid')
# resnet50_unet.summary()

In [ ]:
gc.collect() # Block all the garbage that has been generated

In [ ]:
from keras import models, layers
from keras.applications import ResNet50
from keras.layers import BatchNormalization

# Define upsampling functions
def upsample_conv(filters, kernel_size, strides, padding):
    return layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)

# Set upsampling mode
upsample = upsample_conv

# Input layer
input_img = layers.Input((768, 768, 3), name='RGB_Input')

# Load pre-trained ResNet50
resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_img)

# Extract layers for skip connections
c1 = resnet_base.get_layer('conv1_relu').output
c2 = resnet_base.get_layer('conv2_block3_out').output
c3 = resnet_base.get_layer('conv3_block4_out').output
c4 = resnet_base.get_layer('conv4_block6_out').output
c5 = resnet_base.get_layer('conv5_block3_out').output

# Decoder (Upsampling Path)
u6 = upsample(512, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = layers.concatenate([u6, c4])
c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
c6 = BatchNormalization()(c6)
c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
c6 = BatchNormalization()(c6)

u7 = upsample(256, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = layers.concatenate([u7, c3])
c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
c7 = BatchNormalization()(c7)
c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
c7 = BatchNormalization()(c7)

u8 = upsample(128, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = layers.concatenate([u8, c2])
c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
c8 = BatchNormalization()(c8)
c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
c8 = BatchNormalization()(c8)

u9 = upsample(64, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = layers.concatenate([u9, c1])
c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
c9 = BatchNormalization()(c9)
c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
c9 = BatchNormalization()(c9)

# Ensure the final upsampling restores the original input size
u10 = upsample(32, (2, 2), strides=(2, 2), padding='same')(c9)
c10 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u10)
c10 = BatchNormalization()(c10)
c10 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c10)
c10 = BatchNormalization()(c10)

# Output layer
output_layer = layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)

# Build the model
resnet50_unet = models.Model(inputs=[input_img], outputs=[output_layer])

# Print the model summary
resnet50_unet.summary()


In [ ]:
# Compute dice coefficient, loss with BCE and compile the model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam

# Dice Coefficient
import tensorflow.keras.backend as K

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


# Dice with BCE Loss
def dice_p_bce(y_true, y_pred):
    # Binary Cross-Entropy loss
    bce_loss = binary_crossentropy(y_true, y_pred)
    
    # Dice coefficient loss
    dice_loss = 1 - dice_coef(y_true, y_pred)
    
    # Combined loss
    alpha = 1e-3
    combo_loss = bce_loss + alpha * dice_loss
    
    return combo_loss
# Compile the model
resnet50_unet.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[dice_coef])

In [ ]:
# Preparing Callbacks
# Preparing Callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

# Best model weights
weight_path = "{}_weights.best.weights.h5".format('resnet50_unet')
print(type(weight_path))
# Monitor validation dice coeff and save the best model weights
checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only=True)

# Reduce Learning Rate on Plateau
reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, verbose=1, mode='max', 
                                   min_delta=0.0001, cooldown=2, min_lr=1e-6)

# Stop training once there is no improvement seen in the model
early = EarlyStopping(monitor="val_dice_coef", mode="max", patience=10)

# Callbacks ready
callbacks_list = [checkpoint, early, reduceLROnPlat]
print([type(callback) for callback in callbacks_list])


In [ ]:
# # Finalizing steps per epoch
step_count = min(MAX_TRAIN_STEPS, balanced_train_df.shape[0] // BATCH_SIZE)

# Final augmented data being used in training
aug_gen = create_aug_gen(make_image_gen(balanced_train_df))

# Save loss history while training
loss_history = [resnet50_unet.fit(
                                        aug_gen, 
                                        steps_per_epoch=step_count, 
                                        epochs=NB_EPOCHS, 
                                        validation_data=(valid_x, valid_y),
                                        callbacks=callbacks_list,
                                        )]



In [ ]:
# Save the weights to load it later for test data 
# resnet50_unet.load_weights("C:/Users/krish/Downloads/resnet50_unet_weights.best.weights.h5")
resnet50_unet.save('resnet50_unet.h5')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import tensorflow as tf
import cv2

# Define image dimensions and path to the test directory
IMG_HEIGHT = 768  # Change this to your image height
IMG_WIDTH = 768   # Change this to your image width
TEST_DIR = r'E:/test_v2'  # Use raw string literal for Windows paths
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, confusion_matrix
import seaborn as sns

# Function to plot ROC curve
def plot_roc_curve(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

y_true = valid_y.flatten()
y_pred = resnet50_unet.predict(valid_x).flatten()
# Plot ROC curve
plot_roc_curve(y_true, y_pred)




In [ ]:
# Function to plot Precision-Recall curve
def plot_precision_recall_curve(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    
    plt.figure()
    plt.plot(recall, precision, color='b', lw=2, label='Precision-Recall curve')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.show()

# Function to plot Confusion Matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred > 0.5)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()

# Flatten predictions and true labels
y_true = valid_y.flatten()
y_pred = resnet50_unet.predict(valid_x).flatten()
plot_precision_recall_curve(y_true, y_pred)

# Plot Confusion Matrix
plot_confusion_matrix(y_true, y_pred)


In [ ]:
# Load the trained model with custom objects
custom_objects = {'dice_coef': dice_coef}

# Define image dimensions and path to the test directory
IMG_HEIGHT = 768  # Change this to your image height
IMG_WIDTH = 768   # Change this to your image width
TEST_DIR = r'E:/test_v2'  # Use raw string literal for Windows paths

# Define a function to load and preprocess test images
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image
    return img

# Define a function to post-process the predicted mask to extract bounding boxes
def get_bounding_boxes(prediction, threshold=0.15):
    prediction = prediction[0, :, :, 0]  # Extract the single channel
    prediction = (prediction > threshold).astype(np.uint8)  # Apply threshold to binarize
    contours, _ = cv2.findContours(prediction, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
    return bounding_boxes

# Define a function to display the results
def display_results(image_name, test_image, prediction, bounding_boxes):
    plt.figure(figsize=(18, 6))
    
    plt.subplot(1, 3, 1)
    plt.title(f'Test Image: {image_name}')
    plt.imshow(test_image[0])
    
    plt.subplot(1, 3, 2)
    plt.title('Prediction')
    plt.imshow(prediction[0, :, :, 0], cmap='gray')
    
    plt.subplot(1, 3, 3)
    plt.title('Bounding Boxes')
    img_with_boxes = (test_image[0] * 255).astype(np.uint8)  # Convert back to uint8
    for box in bounding_boxes:
        x, y, w, h = box
        cv2.rectangle(img_with_boxes, (x, y), (x + w, y + h), (255, 0, 0), 2)
    plt.imshow(img_with_boxes)
    
    plt.show()

# Iterate over all images in the test directory and display the results
for image_name in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, image_name)
    
    # Load and preprocess the test image
    test_image = load_and_preprocess_image(image_path)
    
    # Make a prediction
    prediction = resnet50_unet.predict(test_image)
    
    # Get bounding boxes from the prediction
    bounding_boxes = get_bounding_boxes(prediction)
    
    # Display the test image, predicted mask, and bounding boxes
    display_results(image_name, test_image, prediction, bounding_boxes)